In [35]:
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk
import os
import pydicom
from registration_utilities import *

%matplotlib auto

Using matplotlib backend: MacOSX


In [36]:
DATADIR_1 = '/Users/riccardobusetti/Desktop/tmp_processed/pa001/st000/se000'
DATADIR_2 = '/Users/riccardobusetti/Desktop/tmp_processed/pa001/st000/se001'

reader_1 = sitk.ImageSeriesReader()
reader_1.SetOutputPixelType(sitk.sitkFloat32)
dicom_names_1 = reader_1.GetGDCMSeriesFileNames(DATADIR_1)
reader_1.SetFileNames(dicom_names_1)

reader_2 = sitk.ImageSeriesReader()
reader_2.SetOutputPixelType(sitk.sitkFloat32)
dicom_names_2 = reader_2.GetGDCMSeriesFileNames(DATADIR_2)
reader_2.SetFileNames(dicom_names_2)

In [37]:
fixed_image = reader_1.Execute()
#, sitk.sitkFloat32)
print(fixed_image)
moving_image = reader_2.Execute()#, sitk.sitkFloat32)

Image (0x7f8813f29a80)
  RTTI typeinfo:   itk::Image<float, 3u>
  Reference Count: 1
  Modified Time: 180356
  Debug: Off
  Object Name: 
  Observers: 
    none
  Source: (none)
  Source output name: (none)
  Release Data: Off
  Data Released: False
  Global Release Data: Off
  PipelineMTime: 178585
  UpdateMTime: 180355
  RealTimeStamp: 0 seconds 
  LargestPossibleRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [192, 192, 50]
  BufferedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [192, 192, 50]
  RequestedRegion: 
    Dimension: 3
    Index: [0, 0, 0]
    Size: [192, 192, 50]
  Spacing: [1.14583, 1.14583, 15.6]
  Origin: [-105.545, -88.5571, -46.8404]
  Direction: 
0.999962 -0.00174532 -0.00848658
0.000424179 0.988188 -0.153247
0.0086538 0.153238 0.988151

  IndexToPointMatrix: 
1.14579 -0.00199985 -0.132391
0.000486039 1.1323 -2.39065
0.00991581 0.175585 15.4152

  PointToIndexMatrix: 
0.872694 0.000370193 0.0075524
-0.00152319 0.862418 0.133735
-0.000544011 -0.0

In [38]:
moving_to_fixed = sitk.Resample(moving_image, fixed_image)
moving_to_fixed_np = sitk.GetArrayFromImage(moving_to_fixed)
moving_image_np = sitk.GetArrayFromImage(moving_image)
fixed_image_np = sitk.GetArrayFromImage(fixed_image)

plt.imshow(moving_to_fixed_np)
plt.show()

TypeError: Invalid dimensions for image data

In [ ]:
moving_to_fixed_np.shape, moving_image_np.shape

In [ ]:
plt.imshow(moving_to_fixed_np[1,:,:])

In [ ]:
plt.imshow(moving_image_np[10,:,:])


In [ ]:
moving_to_fixed_np.shape

In [ ]:
plt.imshow(fixed_image_np[10,:,:])

In [ ]:
moving_image_np.shape

In [ ]:
initial_transform = sitk.CenteredTransformInitializer(fixed_image, 
                                                      moving_to_fixed, 
                                                      sitk.Euler3DTransform(), 
                                                      sitk.CenteredTransformInitializerFilter.GEOMETRY)

In [ ]:
display_registration_results(fixed_image, moving_to_fixed, initial_transform)

In [ ]:
print(initial_transform)

In [ ]:
registration_method = sitk.ImageRegistrationMethod()

registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)

registration_method.SetInterpolator(sitk.sitkLinear)

registration_method.SetOptimizerAsGradientDescent(learningRate=1.0, numberOfIterations=60)
registration_method.SetOptimizerScalesFromPhysicalShift() 

registration_method.SetInitialTransform(initial_transform, inPlace=False)

registration_method.AddCommand(sitk.sitkStartEvent, metric_start_plot)
registration_method.AddCommand(sitk.sitkEndEvent, metric_end_plot)
registration_method.AddCommand(sitk.sitkIterationEvent, 
                               lambda: metric_plot_values(registration_method))

final_transform_v1 = registration_method.Execute(fixed_image, moving_to_fixed)

In [ ]:
print(final_transform_v1)

In [ ]:
%matplotlib auto

In [ ]:
display_registration_results(fixed_image, moving_to_fixed, final_transform_v1)